In [1]:
%load_ext autoreload
%autoreload 2

import anndata as ad
import matplotlib.pyplot as plt
from mmd_wrapper import mmd_combine
from mmd_wrapper.utilities import alignment_visualize, pairwise_error
import numpy as np
import pandas as pd

from commando import ComManDo

2021-10-22 18:37:03.813470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-22 18:37:03.813501: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


# Read Data

In [2]:
data_mod1 = ad.read_h5ad('OPSCA/openproblems_bmmc_multiome_phase1/openproblems_bmmc_multiome_phase1.censor_dataset.output_mod1.h5ad')
data_mod2 = ad.read_h5ad('OPSCA/openproblems_bmmc_multiome_phase1/openproblems_bmmc_multiome_phase1.censor_dataset.output_mod2.h5ad')

In [3]:
#samples = 8000
X1 = data_mod1.X[:, :5]#[:samples]
X2 = data_mod2.X[:, :5]#[:samples]

# Run Mapping

In [ ]:
commando_out = (
    ComManDo(
        distance_mode='euclidean',
        epoch_pd=1,
        log_pd=1000,
        two_step_num=500,
    )
    .fit_transform([X1, X2])
)

use random seed: 666
Shape of Raw data
Dataset 0: (22463, 5)
Dataset 1: (22463, 5)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 1
Calculating intra-group F #1
Calculating intra-group F #2
Calculating intra-group F #3
Calculating intra-group F #4
Calculating intra-group F #5
Calculating intra-group F #6
Calculating intra-group F #7
Calculating intra-group F #8
Calculating intra-group F #9
Calculating intra-group F #10
Calculating intra-group F #11
Calculating intra-group F #12
Calculating intra-group F #13
Calculating intra-group F #14
Calculating intra-group F #15
Calculating intra-group F #16
Calculating intra-group F #17
Calculating intra-group F #18
Calculating intra-group F #19
Calculating intra-group F #20
Calculating intra-group F #21
Calculating intra-group F #22
Calculating intra-group F #23
Calculating intra-group F #24
Calculating intra-group F #25
Calculating intra-group F #26
Calculating intra-group F #27
Calculating intra-

# Evaluate

In [ ]:
print('ComManDo Pairwise Error: ' + str(pairwise_error(*commando_out, normalize_by_feature=True)))

plt.subplots(figsize=(12, 6))
plt.subplot(1, 2, 1)
alignment_visualize(*commando_out, alg_name='ComManDo')

plt.subplot(1, 2, 2)
plt.axis([-.1, .1, -.1, .1])
alignment_visualize(*commando_out, alg_name='ComManDo (Zoomed)')